
# Glue Studio Notebook
You are now running a **Glue Studio** notebook; before you can start using your notebook you *must* start an interactive session.

## Available Magics
|          Magic              |   Type       |                                                                        Description                                                                        |
|-----------------------------|--------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|
| %%configure                 |  Dictionary  |  A json-formatted dictionary consisting of all configuration parameters for a session. Each parameter can be specified here or through individual magics. |
| %profile                    |  String      |  Specify a profile in your aws configuration to use as the credentials provider.                                                                          |
| %iam_role                   |  String      |  Specify an IAM role to execute your session with.                                                                                                        |
| %region                     |  String      |  Specify the AWS region in which to initialize a session                                                                                                  |
| %session_id                 |  String      |  Returns the session ID for the running session.                                                                                                          |
| %connections                |  List        |  Specify a comma separated list of connections to use in the session.                                                                                     |
| %additional_python_modules  |  List        |  Comma separated list of pip packages, s3 paths or private pip arguments.                                                                                 |
| %extra_py_files             |  List        |  Comma separated list of additional Python files from S3.                                                                                                 |
| %extra_jars                 |  List        |  Comma separated list of additional Jars to include in the cluster.                                                                                       |
| %number_of_workers          |  Integer     |  The number of workers of a defined worker_type that are allocated when a job runs. worker_type must be set too.                                          |
| %worker_type                |  String      |  Standard, G.1X, *or* G.2X. number_of_workers must be set too. Default is G.1X                                                                            |
| %glue_version               |  String      |  The version of Glue to be used by this session. Currently, the only valid options are 2.0 and 3.0 (eg: %glue_version 2.0)                                |
| %security_config            |  String      |  Define a security configuration to be used with this session.                                                                                            |
| %sql                        |  String      |  Run SQL code. All lines after the initial %%sql magic will be passed as part of the SQL code.                                                            |
| %streaming                  |  String      |  Changes the session type to Glue Streaming.                                                                                                              |
| %etl                        |  String      |   Changes the session type to Glue ETL.                                                                                                                   |
| %status                     |              |  Returns the status of the current Glue session including its duration, configuration and executing user / role.                                          |
| %stop_session               |              |  Stops the current session.                                                                                                                               |
| %list_sessions              |              |  Lists all currently running sessions by name and ID.                                                                                                     |
| %spark_conf                 |  String      |  Specify custom spark configurations for your session. E.g. %spark_conf spark.serializer=org.apache.spark.serializer.KryoSerializer                       |

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [ ]:
import datetime
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import explode
end_date = datetime.date.today()

In [ ]:
# Load json file
protocols_dyf = glueContext.create_dynamic_frame.from_options(
    format_options={"multiline": False},
    connection_type="s3",
    format="json",
    connection_options={
        "paths": [f"s3://dataspartan-test-bucket/defillama-analytics/raw-data/protocols/results/llama_protocols_{end_date}.json"],
        "recurse": False,
    },
)

In [ ]:
# Resolve ChoiceType field, with 2 choices - DoubleType & IntType
# Flatten JSON file
protocols_dyfc = protocols_dyf.resolveChoice(choice="make_cols").relationalize("root", "s3://dataspartan-test-bucket/unspecified/temp-dir/")

In [ ]:
protocols_dyfc_root = protocols_dyfc.select('root_protocols')
protocols_df_root =  protocols_dyfc_root.toDF()
new_cols=(column.replace('.', '_') for column in protocols_df_root.columns)
protocols_df_root = protocols_df_root.toDF(*new_cols)

In [ ]:
dtypes_all = protocols_df_root.dtypes

In [ ]:
#Extract all columns names in DF
col_all = []
for col in dtypes_all:
    col_all.append(col[0])

In [ ]:
# Filter out all columns names that were under ChoiceType field 
col_int = []
col_double = []
for col in dtypes_all:
    if '_int' in col[0]:
        key = col[0].replace('_int', '')
        col_int.append(key)
    elif '_double' in col[0]:
        key = col[0].replace('_double', '')
        col_double.append(key)

In [ ]:
# Only include columns names that have both int and double, to confirm that they were from ChoiceType field
col_match = [x for x in col_int if x in col_double]

In [ ]:
# Combine the two lists - Used to drop columns later
col_int_double = []
for col in col_match:
    _int = col+'_int'
    _double = col+'_double'
    col_int_double += [_int,_double]

In [ ]:
# Filter out remaining columns that did not belong to the ChoiceType fields
col_remain =  [x for x in col_all if x not in col_int_double]

In [ ]:
col_fillna = {}
for col in col_int_double:
    col_fillna[col] = '0'

In [ ]:
#fill all ChoiceType field related columns with 0 for to sum later.
protocols_df_root = protocols_df_root.fillna(col_fillna)

In [ ]:
# Combine columns from the same ChoiceType field into one column
for col in col_match:
    _int = '`'+col+'_int`'
    _double = '`'+col+'_double`'
    protocols_df_root = protocols_df_root.withColumn(col, protocols_df_root[_int] + protocols_df_root[_double])
    # protocols_df_root = protocols_df_root.drop(protocols_df_root[_int] + protocols_df_root[_double])

In [ ]:
# Remove ChoiceType related columns
# Remove text-based columns as they cause error when exporting as csv - Solution to be figured out down the road.
protocols_df_root_new = protocols_df_root.drop(*col_int_double, 'protocols_val_audit_note', 'protocols_val_description')

In [ ]:
#https://medium.com/geekculture/aws-glue-simple-job-to-write-part-csv-files-to-a-single-file-d805eddbe641
protocols_df_root_new.repartition(1).write.mode('overwrite').csv(f's3://dataspartan-test-bucket/defillama-analytics/raw-data/protocols/unnested/dump/root_protocols_{end_date}',header = 'true')

In [ ]:
import boto3
client = boto3.client('s3')
BUCKET_NAME= 'dataspartan-test-bucket'
PREFIX_start ='defillama-analytics/raw-data/protocols/unnested/'
PREFIX_end =f'/root_protocols_{end_date}'
response = client.list_objects(
    Bucket=BUCKET_NAME, Prefix=PREFIX_start+'dump'+PREFIX_end,)
#Helps to fetch the file name of the file created in this glue job
name = response['Contents'][0]['Key']
client.copy_object(Bucket=BUCKET_NAME, 
#The csv file generated can be renamed with a name as desired !
CopySource=BUCKET_NAME+'/'+name, Key=PREFIX_start+f'root_protocols/root_protocols_single_file.csv')